1. **Directory**

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

2.**Dataframe**

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/bioprel/data_texturometer.csv')
df

3. **Graph Raw Data**

In [ ]:
import plotly.express as px

# Load the dataset from Kaggle's input directory (adjust the path based on your dataset location)
data = df
# Drop rows with NaN values in the 'Time' and 'Stress' columns
data_cleaned = data.dropna(subset=['Time', 'Stress']).copy()  # Explicitly create a copy

# Create a 'Legend' column combining 'Set' and 'File Name' using .loc for safer indexing
data_cleaned['Legend'] = data_cleaned['Set'] + ":" + data_cleaned['File Name']

# Create an interactive plot using plotly with hover data
fig = px.line(data_cleaned, 
              x='Time', 
              y='Stress', 
              color='Legend',  # Use the combined 'Legend' column for coloring
              hover_data=['Set', 'File Name'],  # Include 'Set' and 'File Name' in the hover data
              title='Stress vs. Time for All Data')

fig.update_traces(mode='lines+markers')
fig.show()


In [ ]:
data_cleaned

4. **Filtering the data at peak stress**

In [ ]:
# Create a 'Legend' column combining 'Set' and 'File Name' for clarity in the plot
data_cleaned['Legend'] = data_cleaned['Set'] + ":" + data_cleaned['File Name']

# Find the time value for peak stress across all data
peak_stress_index = data_cleaned['Stress'].idxmax()
peak_time = data_cleaned.loc[peak_stress_index, 'Time']

# Filter the dataframe to only include values up to and including the global peak time
filtered_df = data_cleaned[data_cleaned['Time'] <= peak_time]

# Create an interactive plot using plotly with hover data
fig = px.line(filtered_df, 
              x='Time', 
              y='Stress', 
              color='Legend',  # Use the 'Legend' column for coloring
              hover_data=['Set'], 
              labels={'Time': 'Time (s)', 'Stress': 'Stress (KPa)'},  # Update axis labels
              title='Stress vs. Time')

fig.update_traces(mode='lines+markers')
fig.show()


In [ ]:
filtered_df

5. **Stress at input value**

In [ ]:
import pandas as pd

# Load the dataset
data_positive = filtered_df

# Ask the user for a strain value
#input_strain_value = float(input("Enter the strain value: "))
input_strain_value = 25

def get_stress_at_strain(group):
    # Find the strain value closest to the user input
    closest_strain = group['Strain'].sub(input_strain_value).abs().idxmin()
    return group.loc[closest_strain, 'Stress']

# Group by the identifiers and apply the function
stress_at_input_strain = data_positive.groupby(['File Name', 'Prefix', 'Group', 'Set']).apply(get_stress_at_strain).reset_index()
stress_at_input_strain.rename(columns={0: f'Stress at {input_strain_value}'}, inplace=True)

# Extract the 'Name' column from 'File Name'
stress_at_input_strain['Name'] = stress_at_input_strain['File Name'].str.split().str[0]

# Reorder columns
stress_at_input_strain = stress_at_input_strain[['File Name', 'Prefix', 'Group', 'Name', 'Set', f'Stress at {input_strain_value}']]

stress_at_input_strain



In [ ]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# Extract unique conditions
conditions = stress_at_input_strain['Prefix'] + " " + stress_at_input_strain['Group'].astype(str)

# Custom order for bars
custom_order = ["PG2 4", "PG2 20", "PG2 37", "SG2 4", "SG2 20", "SG2 37","PG4 4", "PG4 20", "PG4 37","SG4 4", "SG4 20", "SG4 37","PG8FK 4", "PG8FK 20", "PG8FK 37","PG8B2 4", "PG8B2 20", "PG8B2 37","SG8 4", "SG8 20", "SG8 37"]

# Generate rainbow colors based on custom order
colors = plt.cm.rainbow(np.linspace(0, 1, len(custom_order)))
colors = ["rgb" + str(tuple(int(c * 255) for c in color[:3])) for color in colors]
color_mapping = dict(zip(custom_order, colors))

# Create a dictionary for easy access to stress values for each condition
stress_at_input_strain_dict = stress_at_input_strain.groupby(conditions)[f'Stress at {input_strain_value}'].apply(list).to_dict()

# Plot a bar graph of these mean stress values for each condition
fig = go.Figure()

for condition in custom_order:
    if condition in stress_at_input_strain_dict:
        sub_df = stress_at_input_strain[conditions == condition]
        mean_value = sub_df[f'Stress at {input_strain_value}'].mean() / 1000
        std_value = sub_df[f'Stress at {input_strain_value}'].std() / 1000

        fig.add_trace(go.Bar(
            x=[condition],
            y=[mean_value],
            error_y=dict(type='data', array=[std_value], visible=True),
            marker_color=color_mapping[condition],
            name=condition,
            legendgroup=condition  # Set legend group
        ))

        # Individual stress values as dots
        individual_stress_values = (sub_df[f'Stress at {input_strain_value}'] / 1000).tolist()
        file_names = sub_df['File Name'].tolist()
        sets = sub_df['Set'].tolist()

        for value, file_name, set_val in zip(individual_stress_values, file_names, sets):
            hover_text = f"File Name: {file_name}<br>Set: {set_val}<br>Stress: {value:.2f} kPa"
            fig.add_trace(go.Scatter(
                x=[condition],
                y=[value],
                mode='markers',
                marker=dict(color='black', size=6),
                showlegend=False,
                text=hover_text,
                hoverinfo="text",
                legendgroup=condition  # Set legend group to match the bar
            ))

fig.update_layout(
    title=f'Mean Stress Values at Strain Value: {input_strain_value}',
    xaxis_title="Condition",
    yaxis_title="Mean Stress Value (KPa)",  # Changed to kPa
    autosize=False,
    width=1000,
    height=500,
    xaxis=dict(tickangle=-45)
)

fig.show()


In [ ]:
from plotly.offline import plot

plot(fig, filename='/kaggle/working/stress_graph_rainbow.html', auto_open=False)

In [ ]:
import numpy as np
import plotly.graph_objects as go
import seaborn as sns


# Extract unique conditions
conditions = stress_at_input_strain['Prefix'] + " " + stress_at_input_strain['Group'].astype(str)

# Custom order for bars
custom_order = ["PG2 4", "PG2 20", "PG2 37", "SG2 4", "SG2 20", "SG2 37","PG4 4", "PG4 20", "PG4 37","SG4 4", "SG4 20", "SG4 37","PG8FK 4", "PG8FK 20", "PG8FK 37","PG8B2 4", "PG8B2 20", "PG8B2 37","SG8 4", "SG8 20", "SG8 37"]

# Generate colors using the seaborn 'mako' color scale
colors = sns.color_palette("mako", n_colors=len(custom_order)).as_hex()
color_mapping = dict(zip(custom_order, colors))

# Create a dictionary for easy access to stress values for each condition
input_strain_value = 25.0  # Since you provided this value
stress_at_input_strain_dict = stress_at_input_strain.groupby(conditions)[f'Stress at {input_strain_value}'].apply(list).to_dict()

# Plot a bar graph of these mean stress values for each condition
fig = go.Figure()

for condition in custom_order:
    if condition in stress_at_input_strain_dict:
        sub_df = stress_at_input_strain[conditions == condition]
        mean_value = sub_df[f'Stress at {input_strain_value}'].mean() / 1000
        std_value = sub_df[f'Stress at {input_strain_value}'].std() / 1000

        fig.add_trace(go.Bar(
            x=[condition],
            y=[mean_value],
            error_y=dict(type='data', array=[std_value], visible=True),
            marker_color=color_mapping[condition],
            name=condition,
            legendgroup=condition  # Set legend group
        ))

        # Individual stress values as dots
        individual_stress_values = (sub_df[f'Stress at {input_strain_value}'] / 1000).tolist()
        file_names = sub_df['File Name'].tolist()
        sets = sub_df['Set'].tolist()

        for value, file_name, set_val in zip(individual_stress_values, file_names, sets):
            hover_text = f"File Name: {file_name}<br>Set: {set_val}<br>Stress: {value:.2f} kPa"
            fig.add_trace(go.Scatter(
                x=[condition],
                y=[value],
                mode='markers',
                marker=dict(color='black', size=6),
                showlegend=False,
                text=hover_text,
                hoverinfo="text",
                legendgroup=condition  # Set legend group to match the bar
            ))

fig.update_layout(
    title=f'Mean Stress Values at Strain Value: {input_strain_value}',
    xaxis_title="Condition",
    yaxis_title="Mean Stress Value (kPa)",  # Changed to kPa
    autosize=False,
    width=1000,
    height=500,
    xaxis=dict(tickangle=-45)
)

fig.show()


6. **Plot the slope**

In [ ]:
import pandas as pd
import numpy as np

#input_strain_value_s = float(input("Enter the strain value: "))

input_strain_value_s = 3

# Filter data for strain values between 0 and 3
filtered_data = filtered_df[(filtered_df['Strain'] >= 0) & (filtered_df['Strain'] <= input_strain_value_s)]

# Calculate slope for each unique 'File Name' using a linear regression fit
slopes = {}
for file_name, group in filtered_data.groupby('File Name'):
    if len(group) > 1:  # Ensure we have at least two data points to compute a slope
        slope, _ = np.polyfit(group['Strain'], group['Stress'], 1)
        slopes[file_name] = slope
    else:
        slopes[file_name] = np.nan  # Not enough data for this file

# Create a new dataframe with slope values
slope_df = filtered_df.drop_duplicates(subset='File Name')[['File Name', 'Prefix', 'Group', 'Set']]
slope_df['Slope_0_3'] = slope_df['File Name'].map(slopes)

slope_df

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns

# Assuming the dataframe slope_df is already loaded

# Group by 'Prefix' and 'Group' and calculate the mean and std of the slopes
slope_stats = slope_df.groupby(['Prefix', 'Group'])['Slope_0_3'].agg(['mean', 'std']).reset_index()

# Extract unique conditions for grouping
slope_df['Condition'] = slope_df['Prefix'] + " " + slope_df['Group'].astype(str)

# Custom order for bars
custom_order = ["PG2 4", "PG2 20", "PG2 37", "SG2 4", "SG2 20", "SG2 37","PG4 4", "PG4 20", "PG4 37","SG4 4", "SG4 20", "SG4 37","PG8FK 4", "PG8FK 20", "PG8FK 37","PG8B2 4", "PG8B2 20", "PG8B2 37","SG8 4", "SG8 20", "SG8 37"]

# Generate colors using the seaborn 'mako' color scale based on custom order
colors = sns.color_palette("mako", n_colors=len(custom_order)).as_hex()
color_mapping = dict(zip(custom_order, colors))

# Create a dictionary for easy access to slope values for each condition
slope_dict = slope_df.groupby('Condition')['Slope_0_3'].apply(list).to_dict()

fig = go.Figure()

for condition in custom_order:
    if condition in slope_dict:
        mean_value = slope_stats[slope_stats['Prefix'] + " " + slope_stats['Group'].astype(str) == condition]['mean'].values[0] / 1000
        std_value = slope_stats[slope_stats['Prefix'] + " " + slope_stats['Group'].astype(str) == condition]['std'].values[0] / 1000
        
        fig.add_trace(go.Bar(
            x=[condition],
            y=[mean_value],
            error_y=dict(type='data', array=[std_value], visible=True),
            marker_color=color_mapping[condition],
            name=condition,
            legendgroup=condition  # Set legend group
        ))

        # Individual slope values as dots
        sub_df = slope_df[slope_df['Condition'] == condition]
        individual_slope_values = (sub_df['Slope_0_3'] / 1000).tolist()
        original_values = sub_df['Slope_0_3'].tolist()
        file_names = sub_df['File Name'].tolist()
        sets = sub_df['Set'].tolist()
        
        for value, file_name, set_val in zip(individual_slope_values, file_names, sets):
            hover_text = f"File Name: {file_name}<br>Set: {set_val}<br>Slope: {value}"
            fig.add_trace(go.Scatter(
                x=[condition],
                y=[value],
                mode='markers',
                marker=dict(color='black', size=6),
                showlegend=False,
                text=hover_text,
                hoverinfo="text",
                legendgroup=condition  # Set legend group to match the bar
            ))


fig.update_layout(
    title='Slope Values',
    xaxis_title="Condition",
    yaxis_title="Slope (KPa)",
    autosize=False,
    width=1000,
    height=500,
    xaxis=dict(tickangle=-45)
)


fig.show()



In [ ]:
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# Assuming the dataframe slope_df is already loaded

# Group by 'Prefix' and 'Group' and calculate the mean and std of the slopes
slope_stats = slope_df.groupby(['Prefix', 'Group'])['Slope_0_3'].agg(['mean', 'std']).reset_index()

# Extract unique conditions for grouping
slope_df['Condition'] = slope_df['Prefix'] + " " + slope_df['Group'].astype(str)

# Custom order for bars
custom_order = ["PG2 4", "PG2 20", "PG2 37", "SG2 4", "SG2 20", "SG2 37","PG4 4", "PG4 20", "PG4 37","SG4 4", "SG4 20", "SG4 37","PG8FK 4", "PG8FK 20", "PG8FK 37","PG8B2 4", "PG8B2 20", "PG8B2 37","SG8 4", "SG8 20", "SG8 37"]

# Generate rainbow colors based on custom order
colors = plt.cm.rainbow(np.linspace(0, 1, len(custom_order)))
colors = ["rgb" + str(tuple(int(c * 255) for c in color[:3])) for color in colors]
color_mapping = dict(zip(custom_order, colors))

# Create a dictionary for easy access to slope values for each condition
slope_dict = slope_df.groupby('Condition')['Slope_0_3'].apply(list).to_dict()

fig = go.Figure()

for condition in custom_order:
    if condition in slope_dict:
        mean_value = slope_stats[slope_stats['Prefix'] + " " + slope_stats['Group'].astype(str) == condition]['mean'].values[0] / 1000
        std_value = slope_stats[slope_stats['Prefix'] + " " + slope_stats['Group'].astype(str) == condition]['std'].values[0] / 1000
        
        fig.add_trace(go.Bar(
            x=[condition],
            y=[mean_value],
            error_y=dict(type='data', array=[std_value], visible=True, arrayminus=[std_value]),
            marker_color=color_mapping[condition],
            name=condition,
            legendgroup=condition  # Set legend group
        ))

        # Individual slope values as dots
        sub_df = slope_df[slope_df['Condition'] == condition]
        individual_slope_values = (sub_df['Slope_0_3'] / 1000).tolist()
        file_names = sub_df['File Name'].tolist()
        sets = sub_df['Set'].tolist()
        
        for value, file_name, set_val in zip(individual_slope_values, file_names, sets):
            hover_text = f"File Name: {file_name}<br>Set: {set_val}<br>Slope: {value}"
            fig.add_trace(go.Scatter(
                x=[condition],
                y=[value],
                mode='markers',
                marker=dict(color='black', size=6),
                showlegend=False,
                text=hover_text,
                hoverinfo="text",
                legendgroup=condition  # Set legend group to match the bar
            ))

fig.update_layout(
    title='Slope Values',
    xaxis_title="Condition",
    yaxis_title="Slope (KPa)",
    autosize=False,
    width=1000,
    height=500,
    xaxis=dict(tickangle=-45)
)


fig.show()


In [ ]:
plot(fig, filename='/kaggle/working/slope_graph_rainbow.html', auto_open=False)